In [ ]:
# # python version utilised Python 3.10.6


# %pip install yfinance==0.2.18 -q
# %pip install numpy==1.21.5
# %pip install scipy==1.9.3
# %pip install scipy==1.4.2


In [9]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, date

In [10]:
def remove_dot(input: str) -> str:
    if '.' in input:
        return input.split('.')[0]
    else:
        return input

# Assets   

In [11]:
assets_dict = {
    "AUBANK.NS": "AU Small Finance Bank",
    "AXISBANK.NS": "Axis Bank",
    "BANDHANBNK.NS": "Bandhan Bank",
    "BANKBARODA.NS": "Bank of Baroda Ltd",
    "FEDERALBNK.NS": "Federal Bank",
    "HDFCBANK.NS": "HDFC Bank",
    "ICICIBANK.NS": "ICICI Bank",
    "IDFCFIRSTB.NS": "IDFC First Bank",
    "INDUSINDBK.NS": "IndusInd Bank",
    "KOTAKBANK.NS": "Kotak Mahindra Bank",
    "PNB.NS": "Punjab National Bank",
    "SBIN.NS": "SBI"
}

assets_list = [key.replace('.NS', '') for key in list(assets_dict.keys())]
holdings = [(i+1) * 100 for i in range(len(assets_dict))]
 

Downloading data from the  yfinance

In [12]:

import yfinance as yf

hist_data = pd.DataFrame()
assets_tickers = list(assets_dict.keys())

for tick in assets_tickers:
    ticker = yf.Ticker(tick)
    hist_data[remove_dot(tick)] = yf.download(ticker.ticker, start="2019-01-01", end="2023-01-01")['Adj Close']
    
display(hist_data.head(10))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,AUBANK,AXISBANK,BANDHANBNK,BANKBARODA,FEDERALBNK,HDFCBANK,ICICIBANK,IDFCFIRSTB,INDUSINDBK,KOTAKBANK,PNB,SBIN
Date,,,,,,,,,,,,
2019-01-01,310.359650,625.569641,551.757385,119.695847,89.456673,1048.267578,358.771851,43.349998,1566.793701,1248.404419,78.083298,292.297607
2019-01-02,309.511322,618.339661,547.247803,115.861305,91.188095,1038.702759,359.610229,42.700001,1548.136719,1238.570679,76.175240,286.736511
2019-01-03,310.060242,606.273071,529.060730,115.812775,88.494774,1030.553101,358.278748,43.049999,1531.934570,1233.229248,76.175240,284.004791
2019-01-04,311.307770,617.890869,524.204224,118.142609,91.188095,1033.310303,360.202057,45.150002,1520.249268,1245.908569,79.551025,290.395111
2019-01-07,307.565186,635.691711,493.628082,119.113380,90.995705,1034.871826,362.667786,46.150002,1531.001709,1244.560669,78.817162,289.078033
2019-01-08,310.908569,649.104614,472.814545,120.229767,91.620941,1026.258789,374.947388,46.000000,1549.265869,1226.889648,79.942413,297.858643
2019-01-09,314.800842,668.251648,451.356750,119.356071,93.208076,1032.993164,377.018677,46.250000,1572.832642,1236.224365,78.963928,297.858643
2019-01-10,325.230164,661.420532,467.363373,119.841461,92.005707,1028.942749,374.306335,46.950001,1537.629883,1217.554932,79.502098,298.102570
2019-01-11,325.429810,664.661560,450.365631,117.608696,91.765236,1030.601807,373.369324,46.700001,1487.796021,1221.198853,78.915009,295.370819


In [13]:
# calculate historical log returns
hist_return = np.log(hist_data / hist_data.shift())
hist_return = hist_return.dropna()

port_cov = hist_return.cov()             # portfolio covariance matrix
port_corr = hist_return.corr()           # portfolio correlation matrix

In [15]:
V_i = hist_data.iloc[-1] * holdings        # dollar value as of end_date
V_i = V_i.values                       # convert to vector
V_p = V_i.sum()                          # dollar value of the portfolio

In [17]:
z = norm.ppf(0.95, 0, 1)                # z value
sigma_p = np.sqrt(np.dot(V_i.T, np.dot(port_cov.values,V_i)))    # note it's in dollar amount
VaR_p = z * sigma_p                      # portfolio VaR

In [18]:
sigma_i = np.sqrt(np.diag(port_cov.values))        # individual asset
VaR_i = z * sigma_i * V_i

cov_ip = np.dot(port_cov.values, V_i)/V_p               # covariance
beta_i = cov_ip / (sigma_p*sigma_p/V_p/V_p)                  # beta
MVar_i = VaR_p/V_p*beta_i                                    # marginal var

In [19]:
CVaR_i = MVar_i * V_i                                        # component var
CVaR_i_df = pd.DataFrame(data=np.column_stack((V_i, V_i/V_p, CVaR_i, CVaR_i/VaR_p, beta_i)))
CVaR_i_df.index = assets_list
CVaR_i_df.columns = ['Position ($)', 'Position (%)','CVaR ($)','CVaR (%)', 'Beta']
print(CVaR_i_df)

            Position ($)  Position (%)      CVaR ($)  CVaR (%)      Beta
AUBANK      6.544500e+04      0.011211   1457.560357  0.008090  0.721626
AXISBANK    1.867500e+05      0.031991   6102.156941  0.033869  1.058729
BANDHANBNK  7.026000e+04      0.012036   2417.494258  0.013418  1.114857
BANKBARODA  7.428000e+04      0.012724   2075.279228  0.011519  0.905246
FEDERALBNK  6.952500e+04      0.011910   2282.183563  0.012667  1.063583
HDFCBANK    9.768900e+05      0.167343  23204.512588  0.128794  0.769642
ICICIBANK   6.235950e+05      0.106823  19367.523689  0.107497  1.006314
IDFCFIRSTB  4.704000e+04      0.008058   1373.620895  0.007624  0.946153
INDUSINDBK  1.098090e+06      0.188104  50523.117544  0.280423  1.490782
KOTAKBANK   1.827250e+06      0.313011  48463.010745  0.268988  0.859359
PNB         6.209500e+04      0.010637   1484.718230  0.008241  0.774728
SBIN        7.364400e+05      0.126153  21416.462414  0.118870  0.942263
